In [1]:
#Import library Pandas to work on data frames
import pandas as pd
import chardet

In [2]:
with open(r"E:\Aegis\Project\Data_set\rechatterbotpdf/ED_1.1.csv", 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

Aegis_Domain= pd.read_csv(r"E:\Aegis\Project\Data_set\rechatterbotpdf/ED_1.1.csv", encoding=result['encoding'])
Data_Mining_General=pd.read_csv(r"E:\Aegis\Project\Data_set\rechatterbotpdf/2.csv", encoding=result['encoding'])
Pythom_R_ML=pd.read_csv(r"E:\Aegis\Project\Data_set\rechatterbotpdf/3.csv", encoding=result['encoding'])
Statistics=pd.read_csv(r"E:\Aegis\Project\Data_set\rechatterbotpdf/4.csv", encoding=result['encoding'])
Hadoop=pd.read_csv(r"E:\Aegis\Project\Data_set\rechatterbotpdf/5.csv", encoding=result['encoding'])


##
#Size of each data set
print(Aegis_Domain.shape)
print(Data_Mining_General.shape)
print(Pythom_R_ML.shape)
print(Statistics.shape)
print(Hadoop.shape)


(430, 1)
(48, 1)
(98, 1)
(100, 1)
(198, 1)


In [3]:
#Pre processing the data
import re
def cleaning_text(review):
    cleaned =re.sub("[^a-zA-Z/:.?]", " ",review)
    words =cleaned.split()
    return(" ".join( words ))
new_words=[]
for i in range(0,Aegis_Domain.size):
    new_words.append(cleaning_text(Aegis_Domain["QA"][i]))
for i in range(0,Data_Mining_General.size):
    new_words.append(cleaning_text(Data_Mining_General["QA"][i]))
for i in range(0,Pythom_R_ML.size):
    new_words.append(cleaning_text(Pythom_R_ML["QA"][i]))
for i in range(0,Statistics.size):
    new_words.append(cleaning_text(Statistics["QA"][i]))
for i in range(0,Hadoop.size):
    new_words.append(cleaning_text(Hadoop["QA"][i]))



In [4]:
df = pd.DataFrame(new_words, columns=['QA'])
print(df.shape)
print(df.size)
#print df.tail

(874, 1)
874


In [5]:
from nltk import corpus
from chatterbot import ChatBot
#import logging
from chatterbot.trainers import ChatterBotCorpusTrainer

# Uncomment the following line to enable verbose logging
# logging.basicConfig(level=logging.INFO)

# Create a new instance of a ChatBot
chatbot2 = ChatBot("M_University_ChatBot", 
    storage_adapter="chatterbot.storage.JsonFileStorageAdapter",
    logic_adapters=[
        "chatterbot.logic.MathematicalEvaluation",
        "chatterbot.logic.TimeLogicAdapter",
        "chatterbot.logic.BestMatch"
    ],
    input_adapter="chatterbot.input.TerminalAdapter",
    output_adapter="chatterbot.output.TerminalAdapter",
    database="../database10.db", read_only=True
)


#Import List trainer for training the chat history
#
from chatterbot.trainers import ListTrainer
#
#Move the preprocessed chat history to conversations
#
conversation=new_words
chatbot2.set_trainer(ListTrainer)
#
#training the conversation
#
chatbot2.train(conversation)
chatbot2.set_trainer(ChatterBotCorpusTrainer)

chatbot2.train(
    "chatterbot.corpus.english"
)

C:\Users\Ajith Vajrala\Anaconda3\lib\site-packages\chatterbot\storage\jsonfile.py:26: UnsuitableForProductionWarning: The JsonFileStorageAdapter is not recommended for production environments.
  self.UnsuitableForProductionWarning


In [6]:
#_____Sentiment Analysis ________#
with open(r"E:\Aegis\Project\Data_set\rechatterbotpdf/chat_sentiment.csv", 'rb') as f:
    result = chardet.detect(f.read()) 

import pandas as pd
df1 = pd.read_csv(r"E:\Aegis\Project\Data_set\rechatterbotpdf/chat_sentiment.csv", encoding=result['encoding'])

from bs4 import BeautifulSoup  
import nltk
import re
from nltk.corpus import stopwords


##
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

print(df1.columns)

clean_review = review_to_words( df1["Questions"][0] )
print(clean_review)


num_reviews = df1["Questions"].size
clean_train_reviews = []

for i in range( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( df1["Questions"][i] ) )
    
    
from sklearn.feature_extraction.text import CountVectorizer

 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

vocab = vectorizer.get_feature_names()
#print vocab

print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, df1["Sentiment"] )



########
clean_test_reviews = [] 
clean_review = review_to_words("what is eligibility criteria" )
clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)
#print result

Index(['Questions', 'Sentiment'], dtype='object')
business analytics


C:\Users\Ajith Vajrala\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Ajith Vajrala\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Training the random forest...


C:\Users\Ajith Vajrala\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Ajith Vajrala\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
#____Text classification____#
import nltk
from nltk.stem.lancaster import LancasterStemmer
#Word stemmer
stemmer = LancasterStemmer()

In [8]:
# Step2
# provide 3 classes of training data
training_data=[]
training_data.append({"class":"greeting", "sentence":"how are you?"})
training_data.append({"class":"greeting", "sentence":"how is your day?"})
training_data.append({"class":"greeting", "sentence":"good day"})
training_data.append({"class":"greeting", "sentence":"how is it going today?"})
training_data.append({"class":"greeting", "sentence":"hi Good morning?"})
training_data.append({"class":"greeting", "sentence":"nice thank you"})
training_data.append({"class":"greeting", "sentence":"its okay fine"})
training_data.append({"class":"greeting", "sentence":"Welcome"})
##
training_data.append({"class":"Aegis", "sentence":"Where is Aegis located?"})
training_data.append({"class":"Aegis", "sentence":"How Aegis related to IBM?"})
training_data.append({"class":"Aegis", "sentence":"What is Aegis?"})
#
#
training_data.append({"class":"Study Loan", "sentence":"Will I get study loan?"})
training_data.append({"class":"Study Loan", "sentence":"Can I get Scholarship"})
training_data.append({"class":"Study Loan", "sentence":"Which bank will provide study loan?"})
#
#
training_data.append({"class":"Faculty", "sentence":"Who will be teaching R?"})
training_data.append({"class":"Faculty", "sentence":"R lecturer?"})
training_data.append({"class":"Faculty", "sentence":"Who will be teaching Python?"})
training_data.append({"class":"Faculty", "sentence":"Python lecturer?"})
training_data.append({"class":"Faculty", "sentence":"faculty for R?"})
training_data.append({"class":"Faculty", "sentence":"faculty for python?"})
training_data.append({"class":"Faculty", "sentence":"faculty for Hadoop?"})
training_data.append({"class":"Faculty", "sentence":"Who will be teaching hadoop?"})
training_data.append({"class":"Faculty", "sentence":"Hadoop lecturer?"})
training_data.append({"class":"Faculty", "sentence":"faculty for Statistics and probability?"})
#
training_data.append({"class":"R Program", "sentence":"Who will be teaching R?"})
training_data.append({"class":"R Program", "sentence":"R lecturer?"})
training_data.append({"class":"R Program", "sentence":"faculty for R?"})
training_data.append({"class":"R Program", "sentence":"What is R programming?"})
training_data.append({"class":"R Program", "sentence":"What is better for Data Analytics Python or R?"})
training_data.append({"class":"R Program", "sentence":"Python or R?"})
training_data.append({"class":"R Program", "sentence":"How course evaluation for R is done?"})
training_data.append({"class":"R Program", "sentence":"R course curriculum?"})
training_data.append({"class":"R Program", "sentence":"Should I have any prior experience for learning R or Python?"})
#
#
training_data.append({"class":"Python", "sentence":"Who will be teaching Python?"})
training_data.append({"class":"Python", "sentence":"Python lecturer?"})
training_data.append({"class":"Python", "sentence":"Faculty for Python?"})
training_data.append({"class":"Python", "sentence":"What is use of Python in data Analytics?"})
training_data.append({"class":"Python", "sentence":"Python course curriculum?"})
training_data.append({"class":"Python", "sentence":"Python or R?"})
training_data.append({"class":"Python", "sentence":"Should I have any prior experience for learning R or Python?"})
training_data.append({"class":"Python", "sentence":"How course evaluation for Python is done?"})
#
#
#
print ("%s sentences of training data" % len(training_data))

41 sentences of training data


In [9]:
# capture unique stemmed words in the training corpus
corpus_words = {}
class_words = {}
# turn a list into a set (of unique items) and then a list again (this removes duplicates)
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    # prepare a list of words within each class
    class_words[c] = []

# loop through each sentence in our training data
for data in training_data:
    # tokenize each sentence into words
    for word in nltk.word_tokenize(data['sentence']):
        # ignore a some things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            # have we not seen this word already?
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1

            # add the word to our words in class list
            class_words[data['class']].extend([stemmed_word])

# we now have each stemmed word and the number of occurances of the word in our training corpus (the word's commonality)
#print ("Corpus words and counts: %s \n" % corpus_words)
# also we have all words in each class
#print ("Class words: %s" % class_words)

In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
esw = set(stopwords.words("english"))

# calculate a score for a given class
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    neat_words = [w for w in word_tokenize(sentence) if w not in esw]
    for word in neat_words:
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with same weight
            score += 1
            
            if show_details:
                print ("   match: %s" % stemmer.stem(word.lower() ))
    return score


In [11]:
def classify(sentence):
    high_class = None
    high_score = 0
    # loop through our classes
    for c in class_words.keys():
        # calculate score of sentence for each class
        score = calculate_class_score(sentence, c, show_details=False)
        # keep track of highest score
        if score > high_score:
            high_class = c
            high_score = score

    return high_class, high_score



In [12]:
print(classify("hi"))
print(classify("Hello"))
print(classify("Good Morning"))
print(classify("Gud Mrng"))
##

('greeting', 1)
(None, 0)
('greeting', 2)
(None, 0)


In [13]:
print(classify("What is aegis?"))
print(classify("What is the class schedule"))
print(classify("is there any internship?"))
print(classify("Internsip??"))
print(classify("Which is better R or python?"))
print(classify("r or python??"))
print(classify("What is this capstone project?"))
print(classify("no idea what capstone is?"))
print(classify("What is the difference between online course?"))


('Aegis', 2)
('Aegis', 1)
(None, 0)
(None, 0)
('R Program', 3)
('Python', 2)
('Aegis', 1)
(None, 0)
('Python', 2)


In [14]:
print(classify("what is hadoop?"))
print(classify("What is logistic Regression?"))
print(classify("HDFS ?"))
print(classify("how do we do indexing in HDFS??"))

('Faculty', 1)
('Aegis', 1)
(None, 0)
(None, 0)


In [15]:
print(classify("What is machine learning?"))
print(classify("Teacher of machine learning?"))
print(classify("how course evaluation is done?"))

('Python', 2)
('Python', 2)
('Python', 3)


In [16]:
print(classify("relation between data science and statistics?"))

('Aegis', 1)


In [ ]:
print("Type something to begin...")

# The following loop will execute each time the user enters input
while True:
    try:
        # We pass None to this method because the parameter
        # is not used by the TerminalAdapter
        bot_input = chatbot2.get_response(None)
        
        clean_test_reviews = [] 
        clean_review = review_to_words(bot_input )
        clean_test_reviews.append( clean_review )

        # Get a bag of words for the test set, and convert to a numpy array
        test_data_features = vectorizer.transform(clean_test_reviews)
        test_data_features = test_data_features.toarray()

        # Use the random forest to make sentiment label predictions
        result = forest.predict(test_data_features)
        print(result)

    # Press ctrl-c or ctrl-d on the keyboard to exit
    except (KeyboardInterrupt, EOFError, SystemExit):
        break

Type something to begin...
Hi


C:\Users\Ajith Vajrala\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Ajith Vajrala\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Hello
['Interested']
How are you babe
I am fine. What about you.
['Interested']
how good is aegis
Check about Aegis at www.aegis.edu.in and www.bellaward.com
['Interested']
should i join aegis or not
Yes it is good to have the basic knowledge and understanding of Java as there is a tool Hadoop that requires the usage of java. However if you don t have any idea of java then Aegis and IBM s faculty will guide you.
['Interested']
